# Master TIDE - Conférences Python 2021

Francis Wolinski

&copy; 2021 Yotta Conseil


# 4. Graphiques : Matplotlib et Seaborn

Il existe de nombreuses librairies graphiques en Python. la librairie de référence est `matplotlib` qui est un framework graphique à la MATLAB.

Nous utiliserons le sous-module `pyplot` et également la libriarie `seaborn` qui en est une extension.

Documentation :
- matplotlib : http://matplotlib.org
- seaborn : http://seaborn.pydata.org

In [ ]:
# import des modules usuels
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# options d'affichage
pd.set_option("display.max_rows", 16)
plt.style.use('seaborn-darkgrid')

## 4.1 Matplotlib.pyplot

[Voir notebook TIDE-04 - matplotlib]

(...)

### 4.1.2 Graphiques standards avec pandas

In [ ]:
# chargement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
geo = geo.sort_index()
geo.head()

In [ ]:
%%time
# chargement de nouvelles données
thd = pd.read_excel('FranceTHD_Open_Data_Observatoire_Juin2015.xlsx',
                        sheet_name="Communes",
                        header=1,
                        index_col="Code INSEE",
                        names=["Département", "Code INSEE", "Commune",
                                "1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                                "DSL 1 Mbit", "DSL 3 Mbit", "DSL 8 Mbit", "DSL 30 Mbit", "DSL 100 Mbit",
                                "Câble 1 Mbit", "Câble 3 Mbit", "Câble 8 Mbit", "Câble 30 Mbit", "Câble 100 Mbit",
                                "Fibre 1 Mbit", "Fibre 3 Mbit", "Fibre 8 Mbit", "Fibre 30 Mbit", "Fibre 100 Mbit"])
thd.head()

**pickle**

Le module pickle implémente des protocoles binaires de sérialisation et dé-sérialisation d'objets Python.

En **pandas**, il existe la méthode `to_pickle()` et la fonction  `read_pickle()`.

A utiliser ponctuellement dans un projet. Attention, ce n'est pas une technique que sauvegarde permanente des data (instable d'une version de pandas à l'autre) !

In [ ]:
# pickle
thd.to_pickle('thd.pkl')

In [ ]:
%%time
thd = pd.read_pickle('thd.pkl')

Un objet *Series* ou *DataFrame* répond à la méthode **plot()** issue de la librairie *matplotlib.pyplot*.

Il existe diférents types de graphiques disponibles :

Option *kind=...*|Type de graphique
-|-
‘line’|line plot (default)
‘bar’|vertical bar plot
‘barh’|horizontal bar plot
‘hist’|histogram
‘box’|boxplot
‘kde’|Kernel Density Estimation plot (estimation par noyau)
‘density’|same as ‘kde’
‘area’|area plot
‘pie’|pie plot
‘scatter’|scatter plot
'hexbin'|hexagone plot

Voir la documentation : http://pandas.pydata.org/pandas-docs/stable/visualization.html

#### Courbes

In [ ]:
# plot élémentaire
thd["Département"].value_counts().plot();

In [ ]:
# line plot du nombre de communes par département
s = thd["Département"].value_counts()

ax = s.plot(use_index=None,
            title="Nombre de communes par département")
ax.legend(["Communes"]);

In [ ]:
# les taux de 1 à 100 Mbit/s dans les Hauts-de-Seine
hds = thd.loc[thd["Département"] == "HAUTS-DE-SEINE", "Commune":"100 Mbit"]
ax = hds.plot(figsize=(8, 4),
             title="Débits dans les Hauts-de-Seine");
ax.set_xlabel("Commune");

<div class="alert alert-success">
<b>Exercice 1</b>
<ul>
    <li>Modifiez le graphique en rajoutant l'option 'subplots=True'.</li>
    <li>Ajoutez les noms des communes en abscisse avec l'option 'xticks' et la méthode 'set_xticklabels()'.</li>
    <li>Orientez les noms des communes à 60°.</li>
</ul>

#### Barres verticales

In [ ]:
# on sélectionne les données de 2 communes avec la méthode isin()
var = thd.loc[thd["Commune"].isin(["Brest", "Rennes"]), "Commune":"100 Mbit"]
var

In [ ]:
# affichage en barres
var.plot(kind='bar');

<div class="alert alert-success">
<b>Exercice 2</b>
<ul>
    <li>Transposez le DataFrame en ne retenant que les colonnes numériques.</li>
    <li>Obtenez le graphique inverse qui compare deux à deux les débits de chaque ville.</li>
    <li>Mettez la légende à droite à l'extérieur du graphique.</li>
</ul>

#### Barres horizontales

In [ ]:
# affichage en barres horizontales
ax = var.plot(kind='barh');
ax.legend(bbox_to_anchor=(1.3, 0.5));

#### Barres empilées

In [ ]:
# débits 30 Mbit des 3 technos pour 2 communes
var = thd.loc[thd["Commune"].isin(["Brest", "Rennes"]),
              ["Commune", "DSL 30 Mbit", "Câble 30 Mbit", "Fibre 30 Mbit"]]
var

In [ ]:
# affichage en barres empilées
ax = var.plot(kind='bar',
              stacked=True,
              color=['c', 'g', 'm'],
              rot=0);
ax.set_xticklabels(var["Commune"])
ax.set_xlabel("Commune")
ax.legend(bbox_to_anchor=(1.4, 0.2));

#### Secteurs

In [ ]:
# affichage en secteurs
geo["Statut"].value_counts().plot(figsize=(5, 5),
                                  kind='pie',
                                  explode=(0.0, 0.1, 0.2, 0.3, 1, 2));

### 6.1.3 Graphiques statistiques

#### Histogrammes
On peut passer en revue les différents histogrammes par débit.

In [ ]:
# histogramme
thd["30 Mbit"].plot(kind='hist');  # 1 Mbit, 3 Mbit, 8 Mbit, 30 Mbit, 100 Mbit

Il est possible de modifier la partition qui vaut 10 par défaut.

In [ ]:
# histogramme
thd["30 Mbit"].plot(kind='hist',
                    bins=5);

#### Boîte à moustaches
Affichage de la médiane, des quartiles et de l'étendue des valeurs.
On peut passer en revue les boites par débit.

In [ ]:
# boite à moustaches du % par débit
thd.loc[:, "1 Mbit":"100 Mbit"].plot(kind='box');

#### Estimateur à noyau
On peut passer en revue les différents noyaux par débit.

In [ ]:
# estimation par noyau du % par débit
thd["8 Mbit"].plot(kind='kde');  # 1 Mbit, 3 Mbit, 8 Mbit, 30 Mbit, 100 Mbit

#### Graphe de retard
Nuage de points d'une séries croisée avec la même série décalée (par défaut de 1). Ce graphe permet de visualiser l'indépendance des valeurs d'une colonne.

On peut afficher les graphes de retard pour les différents débits.

In [ ]:
# graphe de retard par débit
from pandas.plotting import lag_plot
lag_plot(thd["1 Mbit"]);

A noter : la méthode **shift()** permet de décaler un objet *Series* ou *DataFrame*.

In [ ]:
# colonne Superficie
geo["Superficie"].head()

In [ ]:
# colonne Superficie décalée de 1 vers le bas
geo["Superficie"].shift(1).head()

#### Matrices croisées
- Diagonale : histogrammes
- Hors diagonale : nuages de points

On peut tester avec différents départements.

In [ ]:
# matrices croisées
from pandas.plotting import scatter_matrix
var = thd.loc[thd["Département"] == "RHONE"]
scatter_matrix(var[["1 Mbit", "3 Mbit", "8 Mbit"]]);

In [ ]:
# matrices croisées
var = thd[thd["Département"] == "SOMME"]
scatter_matrix(var[["1 Mbit", "3 Mbit", "8 Mbit"]]);

<div class="alert alert-success">
<b>Exercice 3</b>
<ul>
    <li>Créer un objet *Series* avec le nombre de lettres des noms des communes.</li>
    <li>Etablir un histogramme pour cet objet *Series*.</li>
</ul>

### 6.1.4 Nuage de points

On peut obtenir un nuage de points entre l'altitude moyenne et la population des communes. On constate que plus une commune a une altitude élevée moins elle a d'habitants.

In [ ]:
geo.plot(kind='scatter',
       x="Altitude Moyenne",
       y="Population");

### 4.1.5 Cartes (approche naïve)

Puisque nous avons extrait la longitude et la latitude des données par commune. Il est possible de faire un graphique avec les points géométriques de chaque commune.

In [ ]:
geo['Superficie'] /= 100  # en km2
geo['Densité'] = geo['Population'] * 1000 / geo['Superficie']
geo[['Latitude', 'Longitude']] = geo['geo_point_2d'].str.extract('(.*), (.*)').astype(float)
geo.head()

In [ ]:
# nuage de points avec les coordonnées géographiques
geo.plot(kind='scatter', x="Longitude", y="Latitude");

On obtient une carte de France avec les DOM.

<div class="alert alert-success">
<b>Exercice 4</b>
<ul>
    <li>Cherchez une condition pour n'afficher que la France métropolitaine (plusieurs solutions possibles).</li>
</ul>
</div>

In [ ]:
# affichage de metro
metro.plot(kind='scatter', x="Longitude", y="Latitude");

N.B. : Il existe des données cartographiques opendata utilisables en Python.

Voir Basemap : http://matplotlib.org/basemap/users/intro.html

In [ ]:
# zones de couleurs différentes
metro1 = metro.loc[metro["Altitude Moyenne"] <= 1000]
metro2 = metro.loc[metro["Altitude Moyenne"] > 1000]
plt.scatter(metro1["Longitude"], metro1["Latitude"], color='blue')
plt.scatter(metro2["Longitude"], metro2["Latitude"], color='red');

**Remarque**

Il existe dans *matplotlib* (et *seaborn*) un système de palettes de couleurs :

- matplotlib : https://matplotlib.org/stable/tutorials/colors/colormaps.html

- seaborn : http://seaborn.pydata.org/tutorial/color_palettes.html

Les librairies gèrent également :

- Les palettes de couleurs de **ColorBrewer** : http://colorbrewer2.org

- Les 954 couleurs nommées de manière participative par l'initiative **xkcd** : https://xkcd.com/color/rgb/ Voir aussi l'étude https://blog.xkcd.com/2010/05/03/color-survey-results/

L'altitude avec un nuancier bleu.

In [ ]:
# nuancier bleu
plt.scatter(metro["Longitude"],
            metro["Latitude"],
            c=metro["Altitude Moyenne"],
            cmap=plt.cm.Blues,
            edgecolors='none');

En fait, les points sont dessinés dans l'ordre du *DataFrame*. Donc de petites valeurs affichées en clair peuvent venir effacer des valeurs plus grandes affichées en foncé.

On peut corriger le problème en triant préalablement avec la colonne représentant l'intensité de la couleur.

In [ ]:
# nuancier bleu avec tri des valeurs
metro2 = metro.sort_values("Altitude Moyenne")
plt.scatter(metro2["Longitude"],
            metro2["Latitude"],
            c=metro2["Altitude Moyenne"],
            cmap=plt.cm.Blues,
            edgecolors='none');

Utilisation de la palette *Spectral* inversée et ajout d'un nuancier.

N.B. : pour chaque palette, son inverse est obtenu avec le suffixe "*_r*"

In [ ]:
# on utilise plt.cm.Spectral_r
ax = plt.scatter(x=metro2["Longitude"],
            y=metro2["Latitude"],
            c=metro2["Altitude Moyenne"],
            cmap=plt.cm.Spectral_r,
            edgecolors='none')
plt.colorbar(ax);

In [ ]:
# terminaisons fréquentes
res = geo["Commune"].str.extract("(-[^-]+-[^-]+)$", expand=False)
res.dropna().value_counts().head(16)

<div class="alert alert-success">
<b>Exercice 5</b>
<ul>
    <li>Ecrire une fonction qui affiche une carte de France mettant en exergue les communes se terminant par deux mots séparés par des tirets.</li>
    <li>Exemples :</li>
    <ul>
    <li>show(metro, "SUR", "MER")</li>
    <li>show(metro, "SUR", "SEINE")</li>
    <li>show(metro, "LES", "BAINS")</li>
    </ul>
</ul>
</div>

In [ ]:
def show(metro, mot1, mot2):
    pass

show(metro, "SUR", "LOIRE")

<div class="alert alert-success">
<b>Exercice 6</b>
<ul>
    <li>Sélectionner dans 2 sous-dataframes les villes de statut au moins "Préfecture" et les autres villes.</li>
    <li>Afficher les autres villes dans une couleur uniforme.</li>
    <li>Trier les villes de statut au moins "Préfecture" selon la "Population" décroissante et les représenter avec un cercle proportionnel à la "Population' (option *s=...*) et dans une couleur égale à la "Densité".</li>
</ul>
</div>

In [ ]:
from pandas import CategoricalDtype
statuts = ['Commune simple', 'Chef-lieu canton', 'Sous-préfecture',
       'Préfecture', 'Préfecture de région', "Capitale d'état"]
cat_statut = CategoricalDtype(categories=statuts, ordered=True)
geo["Statut"] = geo["Statut"].astype(cat_statut)

<div class="alert alert-success">
<b>Questions</b>
<ul>
    <li>Pourquoi le cercle le plus large représente la ville de Toulouse ?</li>
    <li>Aide : rechercher les Communes qui contiennent le mot "ARRONDISSEMENT".</li>
</ul>
</div>

## 6.2 Seaborn

Seaborn est une extension de matplotlib qui définit une vingtaine de graphiques pour illustrer des distributions, des régressions, des représentations catégorielles, des matrices et des séries temporelles.

Nous allons étudier quelques graphiques.

### 6.2.1 barplot

Show point estimates and confidence intervals as rectangular bars.

In [ ]:
import seaborn as sns

# seaborn barplot
val = thd.loc[thd["Département"] == "VAL-DE-MARNE", "1 Mbit":"100 Mbit"]
g = sns.barplot(data=val, palette="Blues", ci=None)
g.axes.set_title('Tendance centrale des couvertures par débit dans le Val-de-Marne');

### 6.2.2 countplot

Show the counts of observations in each categorical bin using bars.

In [ ]:
# seaborn countplot
geo.sort_values("Commune", inplace=True)
geo["initiale"] = geo["Commune"].apply(lambda x: x[0])
g = sns.countplot(data=geo, x="initiale")
g.axes.set_ylabel('nombre')
g.axes.set_title('Décompte des communes par lettre initiale');

### 6.2.3 distplot + kdeplot

Flexibly plot a univariate distribution of observations.

+ **kdeplot** Fit and plot a univariate or bivariate kernel density estimate.

In [ ]:
# seaborn distplot
sns.displot(x="8 Mbit", data=thd);

In [ ]:
# seaborn distplot
sns.displot(y="8 Mbit", data=thd);

### 6.2.4 distplot + rugplot

Flexibly plot a univariate distribution of observations.

+ **rugplot** Plot datapoints in an array as sticks on an axis.

In [ ]:
# seaborn distplot
var = geo.loc[geo['Département'] == 'HAUTS-DE-SEINE']
sns.displot(x="Superficie", data=var, rug=True);

### 6.2.5 violinplot

Draw a combination of boxplot and kernel density estimate.

In [ ]:
# seaborn violinplot
sns.violinplot(y="Altitude Moyenne", data=geo);

### 6.2.6 regplot

Plot data and a linear regression model fit.

In [ ]:
# seaborn regplot
x = np.random.random(100)
y = x * (1 + np.random.random(100)) / 2
sns.regplot(x=x, y=y);

### 6.2.7 stripplot

Draw a scatterplot where one variable is categorical.

In [ ]:
# seaborn stripplot
sns.stripplot(x="Population", y="Statut", data=geo);

### 6.2.8 heatmap

Plot rectangular data as a color-encoded matrix.

In [ ]:
# seaborn heatmap
var = thd.loc[thd["Département"] == "CANTAL"]
g = sns.heatmap(data=var.loc[:, "1 Mbit":"100 Mbit"], cmap='Blues')
g.set_yticks([])
g.set_ylabel('');

<div class="alert alert-success">
<b>Exercice 7</b>
<ul>
    <li>Afficher une carte de chaleur avec les corrélations du *DataFrame* thd.</li>
    <li>Ajouter des lignes blanches horizontales et verticales séparant les technologies.</li>
</ul>
</div>

In [ ]:
# exercice
# THD sélectionner les communes qui commencent par la lettre "A"
# comprendre et corriger l'erreur